In [7]:
import sys
print(sys.executable)
import matplotlib
print("matplotlib:", matplotlib.__version__)
import QuantLib as ql
print("QuantLib:", ql.__version__)

/Users/lawsonprendergast/miniforge3/envs/lawson-quant/bin/python


ModuleNotFoundError: No module named 'matplotlib'

In [1]:
from lawson_quant_library.parameter import IRCurve, EQVol
from lawson_quant_library.instrument import EQOption
# Use your existing DivCurve import if it lives in parameter.curves
from lawson_quant_library.parameter.curves import DivCurve

REFERENCE_DATE = "2026-01-06"

In [2]:
spot = 100.0

ir_curve = IRCurve.from_deposit_quotes(
    {"1M": 0.0500, "3M": 0.0520, "6M": 0.0530, "1Y": 0.0550},
    reference_date=REFERENCE_DATE,
    name="USD_IR",
)

div_curve = DivCurve(0.00)

vol = EQVol(currency="USD")
vol.set_flat_vol(0.20, reference_date=REFERENCE_DATE)

In [3]:
atm_call = EQOption(
    spot=spot,
    strike=100.0,
    maturity_date="2027-01-06",
    option_type="call",
    ir_curve=ir_curve,
    div_curve=div_curve,
    vol=vol,
)

price = atm_call.price()
delta = atm_call.delta()
vega = atm_call.vega()
iv = atm_call.implied_vol(price, reference_date=REFERENCE_DATE)

price, delta, vega, iv

(10.638477583787322, 0.6434033588546466, 37.288221599511225, 0.2)

In [4]:
call_25 = EQOption(
    spot=spot, strike=105.0, maturity_date="2027-01-06", option_type="call",
    ir_curve=ir_curve, div_curve=div_curve, vol=vol
)
put_25 = EQOption(
    spot=spot, strike=95.0, maturity_date="2027-01-06", option_type="put",
    ir_curve=ir_curve, div_curve=div_curve, vol=vol
)

rr_price = call_25.price() - put_25.price()
rr_delta = call_25.delta() - put_25.delta()

rr_price, rr_delta

(4.578848664559596, 0.8154934682567362)

In [5]:
strikes = [80, 90, 95, 100, 105, 110, 120]
call_prices = []

for k in strikes:
    opt = EQOption(
        spot=spot,
        strike=float(k),
        maturity_date="2027-01-06",
        option_type="call",
        ir_curve=ir_curve,
        div_curve=div_curve,
        vol=vol,
    )
    call_prices.append(opt.price())

list(zip(strikes, call_prices))

[(80, 24.828760090533354),
 (90, 16.925743560003717),
 (95, 13.556011597600339),
 (100, 10.638477583787322),
 (105, 8.184649332459582),
 (110, 6.177857084945037),
 (120, 3.3378861432671827)]